In [1]:
#import os, sys
#from time import time
import torch
from torch import Tensor
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
#from torch.cuda.amp import GradScaler, autocast
#from torch.utils.data._utils.collate import default_collate
#import copy
#from time import time
#import wandb
import matplotlib.pyplot as plt
from typing import Tuple

import sys
sys.path.append('..')
from dataset import AEDataset
from trainer import Trainer, WeakSupervisionTrainer
from model import DualBranchAE
from utils import *
from losses import MSELoss
from pretrainer import PreTrainer
from torchmetrics.classification import BinaryROC
from scipy import ndimage
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression

from tqdm.notebook import tqdm

c:\Users\Golo\miniconda3\envs\BA\Lib\site-packages\torch\utils\_contextlib.py:125: UserWarning: Decorating classes is deprecated and will be disabled in future versions. You should only decorate functions or methods. To preserve the current behavior of class decoration, you can directly decorate the `__init__` method and nothing else.
  warnings.warn("Decorating classes is deprecated and will be disabled in "


In [2]:
# auto reload changes in .py files
%load_ext autoreload
%autoreload 2

In [3]:
# %cd example/

# Dataset

We currently work on the HPC data and within this, we built two different segmentation tasks. Further details are in the paper https://cg.cs.uni-bonn.de/backend/v1/files/publications/torayev-vcbm2020.pdf. Neither the whole dataset nor the model are in this repo. We will set you up once you started your work and give your access.

In [4]:
# which tasks are used is handled by "set". 1 is a binary task for debugging, 2 is multi-class 
# and so is 3 but with asymmetric classes w.r.t. the saggital plane (harder). Details for 
# set 2 and 3 are in the paper.
# 'modality' handles the target provided by __getitem__. Options are reconstruction and segmentation.
# When segmentation is selected, the labels are taken from the annotations attribute. This is also where
# the user-model interacts with the dataset. Ground truth masks are in the label attribute. All other parameters are
# from past experiments and alter the behaviour. This project has been around for a while, so some are not used anymore.

# normalize is usually set to true. Simply normalizes the input. Augment is legacy, we didn't have much success
# with data augmentation. balance takes care of data balancing during a batch. Some classes are under-
# represented so we show them to the model more often. It helps quite a bit during training so consider 
# integrating it. We can talk about how this is done in detail once you start. init defines how the user-model behaves. 
# We considered different behaviours w.r.t. to annotation style and quantity and such. 
# To_gpu moves ALL data to GPU. Since we only work on a single volume (i.e. couple hundred slices) 
# we move everything to GPU and avoid latency in dataloading. Takes a hefty chunk out of the VRAM though 
# but makes things faster.

# Feel free to re-write anything you want. This is partly dated code that could use a re-write anyways.

# Example:
# make a config first. This handles globals and is used through-out the script. Many things that were tried in
# experiments later have not yet made it into the config, but most have.

cfg = {
    # CONFIG
    'name': 'location-unsupervised',
    'project': 'IDVR-localization_pretrain',
    'log': False,
    'rank': 0,
    
    # DATA
    'data_dir': '../../../data/784565/Diffusion/',
    'data_path': '../../../data/784565/Diffusion/data.nii',
    'active_mask_path': '../../../data/784565/Diffusion/nodif_brain_mask.nii.gz',

    #'data_dir': '../../../data/771354/Diffusion/',
    #'data_path': '../../../data/771354/Diffusion/data.nii.gz',
    #'active_mask_path': '../../../data/771354/Diffusion/nodif_brain_mask.nii.gz',
    
    # SELF SUPERVISED PRE-TRAINING
    's_n_epochs': 20,
    's_batch_size': 16, # default: 8
    's_lr': 5e-4, #1e-4, 1e-5        
    
    # TRAINING WITH WEAK SUPERVISION
    'p_n_epochs': 100,
    'w_n_epochs': 10,
    'w_batch_size': 2,
    'w_lr': 5e-4,    #5e-5 
    'w_eval_freq': 100,
    
    # RANDOM FOREST
    'min_samples_leaf': 8,
    
    # USER MODEL
    'init_voxels': 200,
    'refinement_voxels': 200,
    'num_interactions': 10,
    'brush' : False,
    'slice_selection' : 'mean',
    'voxel_selection' : 'max',
}

In [5]:
# we set balance to true. This also effects the dataloader later
balance = False
# dataset = AEDataset(cfg, modality='segmentation', normalize=True,
                    # set=2, augment=False, balance=balance, init='per_class', to_gpu=False)
dataset = AEDataset(cfg, modality='segmentation', normalize=True,
                    set=2, augment=False, balance=balance, init='per_class', to_gpu=False)
# dataset = AEDataset(cfg, modality='segmentation', normalize=True,
                    # set=2, augment=False, balance=balance, init='three_slices', to_gpu=False)

# currently, there are no annotations. We can also enforce this with clear_annotations() at any point
dataset.clear_annotation()
# get initial annotations
annot = dataset.initial_annotation(seed=42)
# and update the dataset
dataset.update_annotation(annot)
print(f"number of annotations: {dataset.annotations.detach().cpu().sum()}")

# The dataset currently always provides 4 items. Input (the image), target (the input for reconstruction or 
# the annotations for segmentation), weights that mask out voxels which are not annotated for segmentation 
# and a brain mask for masking background during reconstruction
item = dataset[0]
print(item.keys())

number of annotations: 1072.0
dict_keys(['input', 'target', 'weight', 'mask'])


In [6]:
for i in range(dataset.label.shape[0]):
    print(dataset.label[i].sum())

tensor(2837632.)
tensor(44468.)
tensor(33859.)
tensor(960.)
tensor(186667.)


In [7]:
print(dataset.label.shape)

torch.Size([5, 145, 145, 145])


In [8]:
print(dataset.annotations.sum())
print(dataset.annotations.sum(dim=0).unique(return_counts=True))

tensor(1072.)
(tensor([0., 1., 2.]), tensor([3047625,     928,      72]))


# Model and Inference

The overall pipeline is illustrated in the README.

In [9]:
# At first, we do not have annotations but still need features for the Random Forest. So we pre-train 
# on a reconstruction task and later re-use the same Encoder (the part of the network that outputs our features),
# simply replace the decoder and resume training. 

# init the model with segmentation decoder. Have a look at the source code for additional guidance. The dataset
# updates the config to contain labels. We initialize with one channel per class.
model = DualBranchAE(encoder    = 'dual',
                     decoder    = 'segmentation',
                     in_size    = 145,
                     n_classes  = len(cfg['labels']),
                     thresholds = 'learned', 
                     dropout = False, 
                     dropout_rate=0.2) #.to(cfg['rank'])

# example model from one of the experiments
#model_path = 'example_dual_xy_0_best.pt'
#model_path = 'models/hope_best.pt'
model_path = 'models/Test_best.pt'
#model_path = 'models/Set3_best.pt'
#model_path = 'models/Dropout-0.5_best.pt'
#model_path = 'models/Dropout-0.2_best.pt'
#model_path = 'models/Dropout-0.05_best.pt'

# load the components
checkpoint           = torch.load(model_path)
model_state_dict     = checkpoint['model_dict']
encoder_state_dict   = {k.replace('encoder.', ''): v for k, v in model_state_dict.items() if 'encoder' in k}
# print(model_state_dict.keys())

# copy encoder weights to model. Decoder weights remain as they are, initialized as random
model.encoder.load_state_dict(encoder_state_dict, strict=True)

# Define the dataloader. If we use balanced sampling in the dataset, we also need the custom balanced_collate 
# function in the dataloader. This handles the unusal batching logic.

if balance:
    loader  = DataLoader(dataset, 
                         batch_size=cfg['w_batch_size'], 
                         shuffle=True, 
                         drop_last=False, 
                         collate_fn=balanced_collate)
else:
    loader  = DataLoader(dataset, 
                         batch_size=16, 
                         shuffle=True, 
                         drop_last=False)

In [10]:
# For evaluation, we are interested in the Random Forest (RF) prediction based on
# the CNN features. 

# write checkpoints for stuff that changes the behaviour of the dataset.
# E.g. balancing changes the __getitem__ method and thus influences 
# evaluation. Turn it off and on later if needed.
augment_checkpoint = dataset.augment
balance_checkpoint = dataset.balance
dataset.augment = False
dataset.balance = False

# define the layer you want the features from. This is usually the encoder output.
f_layer = 'encoder'
# Init the feature extractor. Have a look at PyTorchs Hook functionality.
extractor = FeatureExtractor(model, layers=[f_layer])

# Cache all features for a dataset and reformat/move to numpy for random forest stuff
hooked_results  = extractor(dataset)
features = hooked_results[f_layer]
features  = features.permute(0,2,3,1).numpy()
# In the utils file are a bunch of evaluation scripts, some are not used anymore.
# This one provides F1 scores for the whole dataset based on all ground truth labels
# and also the predictions themselve as given by the RF. We need them later to update the annotations with
# the user model.

# Turn dataset attributes to normal again
dataset.augment = augment_checkpoint
dataset.balance = balance_checkpoint

 #Now you can change the model and features to your liking and try again (e.g. via constrastive learning ;)).
 #The scores from the RF are the signal you need for evaluation, the rest is up to you.

In [11]:
print(features[0,10,10,10])

0.27598205


In [12]:
print(features[0,10,10,10])

0.27598205


In [13]:
print(features.min(), features.max())

-6.903641 22.450981


In [14]:
x = dataset.label.permute(1,2,3,0)
print(dataset.label[:, dataset.brain_mask].sum(dim=1).unique(return_counts=True))
print(x[dataset.brain_mask].sum(dim=0).unique(return_counts=True))
print(x[dataset.brain_mask].sum(dim=1).unique(return_counts=True))
print(x[dataset.brain_mask][:, 1:].sum(dim=1).unique(return_counts=True))

(tensor([   960.,  32601.,  36731., 164087., 557203.]), tensor([1, 1, 1, 1, 1]))
(tensor([   960.,  32601.,  36731., 164087., 557203.]), tensor([1, 1, 1, 1, 1]))
(tensor([1., 2., 3.]), tensor([696485,  47469,     53]))
(tensor([0., 1., 2., 3.]), tensor([557203, 139282,  47469,     53]))


In [15]:
print(x[dataset.brain_mask].shape)
print(x[dataset.brain_mask][:, :1].sum(dim=1).unique(return_counts=True))

torch.Size([744007, 5])
(tensor([0., 1.]), tensor([186804, 557203]))


# Uncertainty Measures

In [16]:
def print_results(n_annots: list, n_voxel:list, f1_scores: list):
    print(f'Iteration | # Annotations | # Annotierte Voxel |F1 Score')
    print(f'----------|---------------|--------------------|--------')
    for i, (n, v, f1) in enumerate(zip(n_annots, n_voxel, f1_scores)):
        if i in [0, 1, 2, 3, 4, 9, 14, 19, 24, 29, 34, 39, 44, 49, 54]:
            print(f'{i+1:>9} | {int(n):>13} | {int(v):>18} | {f1:.3f}')

In [17]:
# wir brauchen für die erste Iteration mindestens 1 Annotation damit der RF funktioniert
def re_init_dataset():
    dataset.clear_annotation()
    annot = dataset.initial_annotation(seed=42)
    dataset.update_annotation(annot)

In [18]:
def train(method: str, n_epochs: int, measures: List[str]):
    re_init_dataset()
    print(f'Selection using {method}')
    #print(f"number of annotations: {dataset.annotations.detach().cpu().sum()}")
    scores, rf_prediction, unc, unc_pc = evaluate_RF(dataset, features, cfg, measures)
    #scores, rf_prediction, unc, unc_pc = evaluate_RF_with_uncertainty(dataset, features, cfg, measures)
    #scores, rf_prediction = evaluate_RF(dataset, features, cfg)
    print(f"Number of initial annotations: {dataset.annotations.detach().cpu().sum()}")
    #print(dataset.annotations.sum(dim=(1,2,3)).unique(return_counts=True))
    print(f'Number of annotated Voxels: {dataset.annotations.any(dim=0).sum()}')
    print(f"Average F1 score for RF after initial user interaction:    {scores['Avg_f1_tracts'].item():.3f}")
    print()
    n_annots = []
    annots = []
    f1_scores = []
    rf_predictions = []
    uncs_pc = []
    uncs = []
    annotated_voxels = []


    uncs_pc.append(unc_pc)


    for i in tqdm(range(n_epochs), desc='User interaction', unit='iteration'):
        #print(f"Iteration {i+1}")
        if method == 'random':
            annot = dataset.refinement_annotation(prediction=rf_prediction, random=True, seed=42)
        elif method == 'totaly-random':
            annot = dataset.refinement_annotation(prediction=rf_prediction, totaly_random = True, seed=42)
        elif method == 'ground-truth':
            annot = dataset.refinement_annotation(prediction=rf_prediction, seed=42)
        else:
            annot = dataset.refinement_annotation(prediction=rf_prediction, uncertainty_map=unc_pc[method], seed=42)
            #annot = dataset.uncertainty_refinement_annotation(prediction=rf_prediction, uncertainty_map=unc_pc[method], seed=42)

        dataset.update_annotation(annot)

        
        annots.append(dataset.annotations.detach().cpu())
        n_annots.append(dataset.annotations.detach().cpu().sum().item())
        annotated_voxels.append(dataset.annotations.any(dim=0).sum().item())
        #print(f"number of annotations: {dataset.annotations.detach().cpu().sum()}")
        scores, rf_prediction, unc, unc_pc = evaluate_RF(dataset, features, cfg, measures)

        #scores, rf_prediction, unc, unc_pc = evaluate_RF_with_uncertainty(dataset, features, cfg, measures)
        #scores, rf_prediction = evaluate_RF(dataset, features, cfg)
        rf_predictions.append(rf_prediction)
        uncs_pc.append(unc_pc)
        uncs.append(unc)
        #print(scores)
        #if i in [0, 1, 2, 3, 4, 9, 14, 19, 24, 29, 34, 39, 44, 49, 54]:
        #    print(torch.any(dataset.annotations, dim=0).sum().item())
        
        if i == 9:
            
            
            if dataset.set == 2:
                print(f'Durschnitt | Andere |  CG  |  CST  |  FX  | CC')
                print(f'-----------|--------|------|-------|------|---')
                print(f"{scores['Avg_f1_tracts'].item():>10.2f} | {scores['Other_f1']:>6.2f} | {scores['CG_f1']:>3.2f} | {scores['CST_f1']:>5.2f} | {scores['FX_f1']:>2.2f} | {scores['CC_f1']:>2.2f}")
                
                print()
                print()

                print(f'Gesamt | Andere |  CG  |  CST  |   FX   | CC')
                print(f'-------|--------|--------|---------|--------|---')
                x = dataset.annotations.sum(dim=(1,2,3))
                print(f"{dataset.annotations.any(dim=0).sum().item():>6} | {x[0].item()} | {x[1].item():>4} | {x[2].item():>5} | {x[3].item()} | {x[4].item()}")

            elif dataset.set == 3:
                print(f'Durschnitt | Andere | IFO-l | IFO-r |  ILF-l | ILF-r | SLF-l | SLF-r ')
                print(f"-----------|--------|-------|-------|--------|-------|-------|--------")
                print(f"{scores['Avg_f1_tracts'].item():>10.2f} | {scores['Other_f1']:>6.2f} | {scores['IFO_left_f1']:>5.2f} | {scores['IFO_right_f1']:>5.2f} | {scores['ILF_left_f1']:>6.2f} | {scores['ILF_right_f1']:>5.2f} | {scores['SLF_left_f1']:>5.2f} | {scores['SLF_right_f1']:>5.2f}")

                print()
                print()

                print(f'Gesamt | Andere | IFO-l | IFO-r | ILF-l | ILF-r | SLF-l | SLF-r ')
                print(f'-------|--------|-------|-------|-------|-------|-------|-------')
                x = dataset.annotations.sum(dim=(1,2,3))
                print(f"{dataset.annotations.any(dim=0).sum().item():>6} | {x[0].item()} | {x[1].item()} | {x[2].item()} | {x[3].item()} | {x[4].item()} | {x[5].item()} | {x[6].item()}")


            print()
            print()
        


        f1_scores.append(scores['Avg_f1_tracts'].item())

        #print(f"Average F1 score for RF after additional user interaction: {scores['Avg_f1_tracts'].item():.4f}")
    
    print_results(n_annots, annotated_voxels, f1_scores)
    return n_annots, annots, f1_scores, rf_predictions, uncs_pc, uncs

### Entropy

In [22]:
ns_e, ans_e, f1s_e, rf_preds_e, uncs_pc_e, uncs_e = train(method='entropy', n_epochs=2, measures=['entropy'])

Selection using entropy


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (5, 364) + inhomogeneous part.

### Spatial Distance

In [185]:
ns_sd, ans_sd, f1s_sd, rf_preds_sd, uncs_pc_sd, uncs_sd = train(method='spatial-distance', n_epochs=10, measures=['spatial-distance'])

Selection using spatial-distance
Number of initial annotations: 1475.0
Number of annotated Voxels: 1384
Average F1 score for RF after initial user interaction:    0.467



User interaction:   0%|          | 0/10 [00:00<?, ?iteration/s]

Durschnitt | Andere | IFO-l | IFO-r |  ILF-l | ILF-r | SLF-l | SLF-r 
-----------|--------|-------|-------|--------|-------|-------|--------
      0.62 |   0.85 |  0.62 |  0.67 |   0.51 |  0.59 |  0.64 |  0.72


Gesamt | Andere | IFO-l | IFO-r | ILF-l | ILF-r | SLF-l | SLF-r 
-------|--------|-------|-------|-------|-------|-------|-------
 12731 | 9559.0 | 697.0 | 794.0 | 251.0 | 306.0 | 625.0 | 1073.0


Iteration | # Annotations | # Annotierte Voxel |F1 Score
----------|---------------|--------------------|--------
        1 |          2523 |               2383 | 0.503
        2 |          3768 |               3580 | 0.534
        3 |          4795 |               4575 | 0.553
        4 |          5839 |               5573 | 0.565
        5 |          7080 |               6768 | 0.577
       10 |         13305 |              12731 | 0.624


### Feature Distance

In [186]:
ns_fd, ans_fd, f1s_fd, rf_preds_fd, uncs_pc_fd, uncs_fd = train(method='feature-distance', n_epochs=10, measures=['feature-distance'])

Selection using feature-distance
Number of initial annotations: 1475.0
Number of annotated Voxels: 1384
Average F1 score for RF after initial user interaction:    0.467



User interaction:   0%|          | 0/10 [00:00<?, ?iteration/s]

Durschnitt | Andere | IFO-l | IFO-r |  ILF-l | ILF-r | SLF-l | SLF-r 
-----------|--------|-------|-------|--------|-------|-------|--------
      0.58 |   0.72 |  0.64 |  0.63 |   0.57 |  0.53 |  0.48 |  0.61


Gesamt | Andere | IFO-l | IFO-r | ILF-l | ILF-r | SLF-l | SLF-r 
-------|--------|-------|-------|-------|-------|-------|-------
 13551 | 10467.0 | 1087.0 | 1120.0 | 247.0 | 168.0 | 357.0 | 782.0


Iteration | # Annotations | # Annotierte Voxel |F1 Score
----------|---------------|--------------------|--------
        1 |          2800 |               2659 | 0.529
        2 |          4221 |               4024 | 0.538
        3 |          5555 |               5293 | 0.536
        4 |          6750 |               6402 | 0.547
        5 |          8065 |               7654 | 0.547
       10 |         14228 |              13551 | 0.577


### Ground Truth

In [187]:
ns, ans, f1s, rf_preds, uncs_pc, uncs = train(method='ground-truth', n_epochs=10, measures=['ground-truth'])

Selection using ground-truth
Number of initial annotations: 1475.0
Number of annotated Voxels: 1384
Average F1 score for RF after initial user interaction:    0.467



User interaction:   0%|          | 0/10 [00:00<?, ?iteration/s]

Durschnitt | Andere | IFO-l | IFO-r |  ILF-l | ILF-r | SLF-l | SLF-r 
-----------|--------|-------|-------|--------|-------|-------|--------
      0.75 |   0.86 |  0.75 |  0.78 |   0.74 |  0.73 |  0.73 |  0.80


Gesamt | Andere | IFO-l | IFO-r | ILF-l | ILF-r | SLF-l | SLF-r 
-------|--------|-------|-------|-------|-------|-------|-------
 15334 | 12144.0 | 918.0 | 1037.0 | 376.0 | 330.0 | 530.0 | 695.0


Iteration | # Annotations | # Annotierte Voxel |F1 Score
----------|---------------|--------------------|--------
        1 |          2940 |               2782 | 0.551
        2 |          4378 |               4177 | 0.595
        3 |          5830 |               5574 | 0.620
        4 |          7287 |               6968 | 0.666
        5 |          8743 |               8361 | 0.683
       10 |         16030 |              15334 | 0.754


### Random

In [188]:
ns_r, ans_r, f1s_r, rf_preds_r, uncs_pc_r, uncs_r = train(method='random', n_epochs=10, measures=['random'])

Selection using random
Number of initial annotations: 1475.0
Number of annotated Voxels: 1384
Average F1 score for RF after initial user interaction:    0.467



User interaction:   0%|          | 0/10 [00:00<?, ?iteration/s]

Durschnitt | Andere | IFO-l | IFO-r |  ILF-l | ILF-r | SLF-l | SLF-r 
-----------|--------|-------|-------|--------|-------|-------|--------
      0.64 |   0.85 |  0.64 |  0.67 |   0.61 |  0.59 |  0.66 |  0.70


Gesamt | Andere | IFO-l | IFO-r | ILF-l | ILF-r | SLF-l | SLF-r 
-------|--------|-------|-------|-------|-------|-------|-------
 15245 | 12677.0 | 707.0 | 602.0 | 197.0 | 188.0 | 531.0 | 722.0


Iteration | # Annotations | # Annotierte Voxel |F1 Score
----------|---------------|--------------------|--------
        1 |          2905 |               2783 | 0.519
        2 |          4322 |               4183 | 0.542
        3 |          5734 |               5582 | 0.569
        4 |          7192 |               6981 | 0.589
        5 |          8611 |               8379 | 0.596
       10 |         15624 |              15245 | 0.645


In [189]:
for _ in range(10):
    ns_r, ans_r, f1s_r, rf_preds_r, uncs_pc_r, uncs_r = train(method='random', n_epochs=10, measures=['random'])

Selection using random
Number of initial annotations: 1475.0
Number of annotated Voxels: 1384
Average F1 score for RF after initial user interaction:    0.467



User interaction:   0%|          | 0/10 [00:00<?, ?iteration/s]

Durschnitt | Andere | IFO-l | IFO-r |  ILF-l | ILF-r | SLF-l | SLF-r 
-----------|--------|-------|-------|--------|-------|-------|--------
      0.63 |   0.88 |  0.63 |  0.66 |   0.55 |  0.55 |  0.65 |  0.71


Gesamt | Andere | IFO-l | IFO-r | ILF-l | ILF-r | SLF-l | SLF-r 
-------|--------|-------|-------|-------|-------|-------|-------
 14949 | 12813.0 | 469.0 | 442.0 | 142.0 | 182.0 | 433.0 | 759.0


Iteration | # Annotations | # Annotierte Voxel |F1 Score
----------|---------------|--------------------|--------
        1 |          2893 |               2783 | 0.498
        2 |          4314 |               4183 | 0.518
        3 |          5739 |               5583 | 0.529
        4 |          6989 |               6825 | 0.543
        5 |          8406 |               8224 | 0.557
       10 |         15240 |              14949 | 0.625
Selection using random
Number of initial annotations: 1475.0
Number of annotated Voxels: 1384
Average F1 score for RF after initial user interactio

User interaction:   0%|          | 0/10 [00:00<?, ?iteration/s]

Durschnitt | Andere | IFO-l | IFO-r |  ILF-l | ILF-r | SLF-l | SLF-r 
-----------|--------|-------|-------|--------|-------|-------|--------
      0.65 |   0.86 |  0.64 |  0.68 |   0.56 |  0.62 |  0.68 |  0.70


Gesamt | Andere | IFO-l | IFO-r | ILF-l | ILF-r | SLF-l | SLF-r 
-------|--------|-------|-------|-------|-------|-------|-------
 15195 | 12695.0 | 554.0 | 601.0 | 211.0 | 175.0 | 605.0 | 731.0


Iteration | # Annotations | # Annotierte Voxel |F1 Score
----------|---------------|--------------------|--------
        1 |          2960 |               2783 | 0.526
        2 |          4324 |               4138 | 0.528
        3 |          5745 |               5538 | 0.553
        4 |          7113 |               6871 | 0.578
        5 |          8534 |               8270 | 0.593
       10 |         15572 |              15195 | 0.646
Selection using random
Number of initial annotations: 1475.0
Number of annotated Voxels: 1384
Average F1 score for RF after initial user interactio

...

Durschnitt | Andere | IFO-l | IFO-r |  ILF-l | ILF-r | SLF-l | SLF-r 
-----------|--------|-------|-------|--------|-------|-------|--------
      0.65 |   0.87 |  0.65 |  0.70 |   0.63 |  0.60 |  0.63 |  0.70


Gesamt | Andere | IFO-l | IFO-r | ILF-l | ILF-r | SLF-l | SLF-r 
-------|--------|-------|-------|-------|-------|-------|-------
 15367 | 13119.0 | 633.0 | 640.0 | 222.0 | 185.0 | 419.0 | 520.0


Iteration | # Annotations | # Annotierte Voxel |F1 Score
----------|---------------|--------------------|--------
        1 |          2914 |               2779 | 0.515
        2 |          4354 |               4178 | 0.544
        3 |          5782 |               5578 | 0.561
        4 |          7212 |               6978 | 0.579
        5 |          8630 |               8376 | 0.601
       10 |         15738 |              15367 | 0.653
Selection using random
Number of initial annotations: 1475.0
Number of annotated Voxels: 1384
Average F1 score for RF after initial user interactio

User interaction:   0%|          | 0/10 [00:00<?, ?iteration/s]

Durschnitt | Andere | IFO-l | IFO-r |  ILF-l | ILF-r | SLF-l | SLF-r 
-----------|--------|-------|-------|--------|-------|-------|--------
      0.64 |   0.85 |  0.65 |  0.69 |   0.58 |  0.60 |  0.65 |  0.70


Gesamt | Andere | IFO-l | IFO-r | ILF-l | ILF-r | SLF-l | SLF-r 
-------|--------|-------|-------|-------|-------|-------|-------
 15133 | 12771.0 | 604.0 | 657.0 | 171.0 | 183.0 | 447.0 | 659.0


Iteration | # Annotations | # Annotierte Voxel |F1 Score
----------|---------------|--------------------|--------
        1 |          2779 |               2661 | 0.519
        2 |          4211 |               4061 | 0.560
        3 |          5644 |               5461 | 0.577
        4 |          7051 |               6853 | 0.592
        5 |          8493 |               8252 | 0.612
       10 |         15492 |              15133 | 0.645
Selection using random
Number of initial annotations: 1475.0
Number of annotated Voxels: 1384
Average F1 score for RF after initial user interactio

User interaction:   0%|          | 0/10 [00:00<?, ?iteration/s]

Durschnitt | Andere | IFO-l | IFO-r |  ILF-l | ILF-r | SLF-l | SLF-r 
-----------|--------|-------|-------|--------|-------|-------|--------
      0.65 |   0.85 |  0.63 |  0.68 |   0.62 |  0.62 |  0.66 |  0.71


Gesamt | Andere | IFO-l | IFO-r | ILF-l | ILF-r | SLF-l | SLF-r 
-------|--------|-------|-------|-------|-------|-------|-------
 15197 | 12803.0 | 617.0 | 624.0 | 221.0 | 215.0 | 426.0 | 680.0


Iteration | # Annotations | # Annotierte Voxel |F1 Score
----------|---------------|--------------------|--------
        1 |          2945 |               2784 | 0.522
        2 |          4271 |               4067 | 0.547
        3 |          5675 |               5466 | 0.576
        4 |          7015 |               6799 | 0.588
        5 |          8466 |               8198 | 0.600
       10 |         15586 |              15197 | 0.652


### Totaly Random

In [25]:
ns_r, ans_r, f1s_r, rf_preds_r, uncs_pc_r, uncs_r = train(method='totaly-random', n_epochs=10, measures=['totaly-random'])


Selection using totaly-random
Number of initial annotations: 1475.0
Average F1 score for RF after initial user interaction:    0.4672



User interaction:   0%|          | 0/10 [00:00<?, ?iteration/s]

0.86
0.66
0.69
0.64
0.62
0.69
0.72
tensor([12558.,   728.,   692.,   278.,   188.,   598.,   780.])
Iteration | # Annotations | Annotations-Verteilung |F1 Score
----------|---------------|------------------------|--------
        1 |          2909 | 0.5365
        2 |          4346 | 0.5701
        3 |          5781 | 0.5944
        4 |          7213 | 0.6134
        5 |          8658 | 0.6290
       10 |         15822 | 0.6694


In [45]:
x = rf_preds[-1] - dataset.label
print(f'Klasse 1: {x[0].unique(return_counts=True)}')
print(f'Klasse 2: {x[1].unique(return_counts=True)}')
print(f'Klasse 3: {x[2].unique(return_counts=True)}')
print(f'Klasse 4: {x[3].unique(return_counts=True)}')
print(f'Klasse 5: {x[4].unique(return_counts=True)}')

Klasse 1: (tensor([-1.,  0.,  1.], dtype=torch.float64), tensor([ 183390, 2861184,    4051]))
Klasse 2: (tensor([-1.,  0.,  1.], dtype=torch.float64), tensor([    582, 3007134,   40909]))
Klasse 3: (tensor([-1.,  0.,  1.], dtype=torch.float64), tensor([    307, 3009528,   38790]))
Klasse 4: (tensor([0., 1.], dtype=torch.float64), tensor([3045455,    3170]))
Klasse 5: (tensor([-1.,  0.,  1.], dtype=torch.float64), tensor([  15093, 2967059,   66473]))


In [92]:
mask = dataset.annotations.any(dim=0)
y = dataset.annotations[:, mask] - dataset.label[:, mask]
z = y.sum(dim=0)
print(z.unique(return_counts=True))

(tensor([-1.,  0.], dtype=torch.float64), tensor([1063, 1567]))


### Logistic Regression

In [37]:
error_maps = [torch.ne(rf_prediction, dataset.label) * 1 for rf_prediction in rf_preds]
error_maps_mean = [torch.any(error_map, dim=0) * 1 for error_map in error_maps]

In [38]:
abc = uncs_pc[0]['entropy'].flatten()
defg = uncs_pc[0]['spatial-distance'].flatten()
hijk = uncs_pc[0]['feature-distance'].flatten()
lmno = torch.stack((abc, defg, hijk), dim=1)
print(lmno.shape)

torch.Size([15243125, 3])


In [39]:
# Create and fit the logistic regression model
for i, (unc_map, error_map) in enumerate(tqdm(zip(uncs_pc, error_maps),total=len(uncs_pc), desc='Calculation', unit='iteration')):
    unc_map_e = unc_map['entropy'].flatten()
    unc_map_sd = unc_map['spatial-distance'].flatten()
    unc_map_fd = unc_map['feature-distance'].flatten()
    logreg_e = LogisticRegression(random_state=0, n_jobs=-1)
    logreg_e.fit(unc_map_e.reshape(-1, 1), error_map.flatten())
    logreg_sd = LogisticRegression(random_state=0, n_jobs=-1)
    logreg_sd.fit(unc_map_sd.reshape(-1, 1), error_map.flatten())
    logreg_fd = LogisticRegression(random_state=0, n_jobs=-1)
    logreg_fd.fit(unc_map_fd.reshape(-1, 1), error_map.flatten())
    print(f"Koeffizient für Iteration {i+1}: Entropy: {logreg_e.coef_}, SD: {logreg_sd.coef_}, FD: {logreg_fd.coef_}")
    logreg_c = LogisticRegression(random_state=0, n_jobs=-1)
    logreg_c.fit(torch.stack((unc_map_e, unc_map_sd, unc_map_fd), dim=1), error_map.flatten())
    print(f"Koeffizient für Iteration {i+1}: Combined: {logreg_c.coef_}")

Calculation:   0%|          | 0/20 [00:00<?, ?iteration/s]

Koeffizient für Iteration 1: Entropy: [[6.80838514]], SD: [[0.03420244]], FD: [[5.14872597]]
Koeffizient für Iteration 1: Combined: [[ 6.72757369 -0.0340217   0.63251322]]
Koeffizient für Iteration 2: Entropy: [[7.02794357]], SD: [[0.03219433]], FD: [[5.32265189]]
Koeffizient für Iteration 2: Combined: [[ 6.73033755 -0.03509393  1.2549075 ]]
Koeffizient für Iteration 3: Entropy: [[6.6982623]], SD: [[0.02620928]], FD: [[5.05152733]]
Koeffizient für Iteration 3: Combined: [[ 6.29420073 -0.0529336   1.9025076 ]]
Koeffizient für Iteration 4: Entropy: [[6.8116268]], SD: [[0.02158613]], FD: [[4.87395059]]
Koeffizient für Iteration 4: Combined: [[ 6.59178342 -0.06249266  1.17257243]]
Koeffizient für Iteration 5: Entropy: [[7.19238669]], SD: [[0.01998846]], FD: [[4.75844355]]
Koeffizient für Iteration 5: Combined: [[ 7.08846542 -0.05720406  0.35012152]]
Koeffizient für Iteration 6: Entropy: [[7.31138677]], SD: [[0.02119565]], FD: [[4.94982176]]
Koeffizient für Iteration 6: Combined: [[ 7.19909